In [29]:
import os
from datetime import datetime
import PIL
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.callbacks import LambdaCallback, CSVLogger
from tensorflow.keras import mixed_precision


In [30]:
csv_logger =CSVLogger('training_log.csv', append=True)

print_callback = LambdaCallback(on_epoch_end=lambda epoch, logs: print(f"Epoch {epoch+1}, Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Val Loss: {logs['val_loss']:.4f}, Val Accuracy: {logs['val_accuracy']:.4f}"))

In [31]:
train_dir = 'data/'
validation_dir = 'data/'

# Image data augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 1129 images belonging to 10 classes.
Found 1129 images belonging to 10 classes.


In [32]:
# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

In [33]:
# Create a new model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') 
])

In [34]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

print_callback = LambdaCallback(on_epoch_end=lambda epoch, logs: 
                                print(f"Epoch {epoch+1}, Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}, Val Loss: {logs['val_loss']:.4f}, Val Accuracy: {logs['val_accuracy']:.4f}"))


In [35]:
# Trenirajte model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[csv_logger, print_callback] 
)

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [ ]:
# Save the model
model.save('bone_fracture_classifier.h5')

In [ ]:
import tensorflow as tf

# Enable mixed precision
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Define a simple model using lower-level TensorFlow APIs
class SimpleModel(tf.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.weights = tf.Variable(tf.random.normal([784, 256], stddev=0.1, dtype=tf.float16))
        self.bias = tf.Variable(tf.zeros([256], dtype=tf.float16))

    def __call__(self, x):
        x = tf.matmul(x, self.weights) + self.bias
        return tf.nn.relu(x)

# Instantiate the model
model = SimpleModel()

# Example input data
x_train = tf.random.normal([32, 784], dtype=tf.float16)  # Batch size 32, input shape 784
y_train = tf.random.uniform([32, 256], maxval=10, dtype=tf.int32)  # Example target

# Loss function
def compute_loss(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf.one_hot(labels, depth=256)))

# Optimizer
optimizer = tf.keras.optimizers.Adam()

# Training step function
@tf.function
def train_step(model, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    gradients = tape.gradient(loss, [model.weights, model.bias])
    optimizer.apply_gradients(zip(gradients, [model.weights, model.bias]))
    return loss

# Training loop
for epoch in range(10):  # Train for 10 epochs
    loss = train_step(model, x_train, y_train)
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")

In [ ]:
import tensorflow

from tensorflow import keras

# Enable mixed precision
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

ImportError: cannot import name 'KerasLazyLoader' from 'tensorflow.python.util.lazy_loader' (/Users/edigraovac/Documents/algebra/Diplomski/2. semestar/Osnove racunalnog vida/project/.venv/lib/python3.9/site-packages/tensorflow/python/util/lazy_loader.py)